# OCR em vídeos - com Tesseract e EAST

In [ ]:
# Baixar imagens para o laboratório.
!wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/imagens.zip
!unzip -n -q imagens.zip

# Baixar fontes para o laboratório
!wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/fontes.zip
!unzip -n -q fontes.zip

# Importando as bibliotecas

In [ ]:
import cv2
import imutils
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from imutils.object_detection import non_max_suppression
from PIL import Image
from PIL import ImageFont, ImageDraw, Image

# Conectando com o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Configuração do Tesseract

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!mkdir tessdata
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true

import pytesseract

In [ ]:
config_tesseract = "--tessdata-dir tessdata --psm 7"

In [ ]:
def tesseract_OCR(img, config_tesseract):
  texto = pytesseract.image_to_string(img, lang='por', config=config_tesseract)
  return texto

# Pré-processamento

Funções para pre-processar as imagens (ROI) extraídas do EAST, desse modo fica mais adequado antes de passar para o Tesseract

In [ ]:
def pre_processamento(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  maior = cv2.resize(gray, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
  valor, otsu = cv2.threshold(maior, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
  return otsu

# Configuração do EAST

Carrega o `frozen_east_text_detection.pb`

In [ ]:
!cp /content/gdrive/MyDrive/Cursos/OCR/frozen_east_text_detection.pb ./
!cp -R /content/gdrive/MyDrive/Cursos/OCR/imagens imagens/
!cp -R /content/gdrive/MyDrive/Cursos/OCR/fontes fontes/

In [ ]:
detector = "frozen_east_text_detection.pb"
largura_EAST, altura_EAST = 640, 640 # 320x320

In [ ]:
min_conf_EAST = 0.9 #@param {type:"number", min:0.0, max:1.0}

In [ ]:
camadas_EAST = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

# Funções EAST

In [ ]:
def dados_geometricos(geometry, y):
  x0_data = geometry[0, 0, y]
  x1_data = geometry[0, 1, y]
  x2_data = geometry[0, 2, y]
  x3_data = geometry[0, 3, y]
  data_angulos = geometry[0, 4, y]

  return data_angulos, x0_data, x1_data, x2_data, x3_data

In [ ]:
def calculos_geometria(data_angulos, x0_data, x1_data, x2_data, x3_data, x, y):
  (offsetX, offsetY) = (x * 4.0, y * 4.0)

  angulo = data_angulos[x]
  cos = np.cos(angulo)
  sin = np.sin(angulo)

  h = x0_data[x] + x2_data[x]
  w = x1_data[x] + x3_data[x]

  fimX = int(offsetX + (cos * x1_data[x]) + (sin * x2_data[x]))
  fimY = int(offsetY - (sin * x1_data[x]) + (cos * x2_data[x]))
  inicioX = int(fimX - w)
  inicioY = int(fimY - h)

  return inicioX, inicioY, fimX, fimY

# Função para processamento da imagem

In [ ]:
def EAST_processamento(img, largura, altura, net, nomes_camadas, min_confianca):
  original = img.copy()
  (H, W) = img.shape[:2]

  proporcao_W = W / float(largura)
  proporcao_H = H / float(altura)

  img = cv2.resize(img, (largura, altura))
  (H, W) = img.shape[:2]

  blob = cv2.dnn.blobFromImage(img, 1.0, (W, H), swapRB=True, crop=False)

  net.setInput(blob)
  (scores, geometry) = net.forward(nomes_camadas)

  (linhas, colunas) = scores.shape[2:4]

  caixas = []
  confiancas = []

  for y in range(0, linhas):
    data_scores = scores[0, 0, y]

    data_angulos, x0_data, x1_data, x2_data, x3_data = dados_geometricos(geometry, y)

    for x in range(0, colunas):
      if data_scores[x] < min_confianca:
        continue

      inicioX, inicioY, fimX, fimY = calculos_geometria(data_angulos, x0_data, x1_data, x2_data, x3_data, x, y)
      confiancas.append(data_scores[x])
      caixas.append((inicioX, inicioY, fimX, fimY))

  return proporcao_W, proporcao_H, confiancas, caixas

# Função para escrita no video

In [ ]:
fonte = './Fontes/calibri.ttf'

In [ ]:
def escreve_texto(texto, x, y, img, fonte, cor=(50, 50, 255), tamanho=22):
  fonte = ImageFont.truetype(fonte, tamanho)
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text((x, y-tamanho), texto, font = fonte, fill = cor)
  img = np.array(img_pil)

  return img

In [ ]:
def fundo_texto(texto, x, y, img, fonte, tamanho=32, cor_fundo=(200, 255, 0)):
  fundo = np.full((img.shape), (0,0,0), dtype=np.uint8)
  texto_fundo = escreve_texto(texto, x, y, fundo, fonte, (255,255,255), tamanho=tamanho)
  texto_fundo = cv2.dilate(texto_fundo,(np.ones((3,5),np.uint8)))
  fx,fy,fw,fh = cv2.boundingRect(texto_fundo[:,:,2])
  cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), cor_fundo, -1)

  return img

# Carrega modelo EAST

In [ ]:
EASTnet = cv2.dnn.readNet(detector)

# Carrega o arquivo de vídeo

In [ ]:
!cp -R /content/gdrive/MyDrive/Cursos/OCR/videos videos/

In [ ]:
arquivo_video = '/content/videos/videoteste02.mp4'
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()
print(conectado, video)

In [ ]:
video.shape

In [ ]:
video_largura = video.shape[1]
video_altura = video.shape[0]

# Redimensionamento do tamanho do video (opcional)

In [ ]:
def redimensionar(largura, altura, largura_maxima = 600):
  if largura > largura_maxima:
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura
  return video_largura, video_altura

In [ ]:
video_largura, video_altura = redimensionar(video.shape[1], video.shape[0], 800)
print(video_largura, video_altura)

# Definindo as configurações do vídeo

In [ ]:
nome_arquivo = 'resultado_east_tesseract.avi'

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')

FourCC é um código de 4 bytes usado para especificar o codec de vídeo. A lista de códigos disponíveis pode ser encontrada no site fourcc.org
* Codecs mais usados: XVID, MP4V, MJPG, DIVX, X264...
* Por exemplo, para salvar em formato mp4 utiliza-se o codec mp4v (o nome do arquivo também precisa possuir a extensão .mp4)

 fourcc = cv2.VideoWriter_fourcc(*'mp4v')

Mais exemplos de outras configurações com o fourcc que é possível usar: https://www.programcreek.com/python/example/89348/cv2.VideoWriter_fourcc

In [ ]:
fps = 24

In [ ]:
saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

# Definindo as variáveis

In [ ]:
amostras_exibir = 20
amostra_atual = 0

In [ ]:
margem = 4

# Processamento do vídeo e exibição do resultado

In [ ]:
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()

  if not conectado:
    break

  frame = cv2.resize(frame, (video_largura, video_altura))

  imagem_cp = frame.copy()

  proporcao_W, proporcao_H, confiancas, caixas = EAST_processamento(frame, largura_EAST, altura_EAST, EASTnet, camadas_EAST, min_conf_EAST)
  deteccoes = non_max_suppression(np.array(caixas), probs=confiancas)
  for (inicioX, inicioY, fimX, fimY) in deteccoes:
    inicioX = int(inicioX * proporcao_W)
    inicioY = int(inicioY * proporcao_H)
    fimX = int(fimX * proporcao_W)
    fimY = int(fimY * proporcao_H)

    cv2.rectangle(frame, (inicioX, inicioY), (fimX, fimY), (200,255,0), 2)

    roi = imagem_cp[inicioY - margem:fimY + margem, inicioX - margem:fimX + margem]

    img_process = pre_processamento(roi)

    texto = tesseract_OCR(img_process, config_tesseract)
    # http://www.asciitable.com/
    texto = ''.join([c if ord(c) < 128 else '' for c in texto]).strip()

    frame = fundo_texto(texto, inicioX, inicioY, frame, fonte, 20, (200,255,0))
    frame = escreve_texto(texto, inicioX, inicioY, frame, fonte, (0,0,0), 20)

  if amostra_atual <= amostras_exibir:
    cv2_imshow(frame)
    amostra_atual += 1

  saida_video.write(frame)

print('Terminou!')
saida_video.release()
cv2.destroyAllWindows()

# OCR em vídeos com EasyOCR

In [ ]:
!pip install easyocr

In [ ]:
from easyocr import Reader
import cv2
from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image
import numpy as np

In [ ]:
lista_idiomas = "en,pt"
idiomas = lista_idiomas.split(",")
print(idiomas)

gpu = True #@param {type:"boolean"}
fonte = './Fontes/calibri.ttf' #@param {type:"string"}

In [ ]:
arquivo_video = "videos/videoteste02.mp4"
cap = cv2.VideoCapture(arquivo_video)

conectado, video = cap.read()
video_largura = video.shape[1]
video_altura = video.shape[0]

In [ ]:
video_largura, video_altura = redimensionar(video.shape[1], video.shape[0], 800)
print(video_largura,video_altura)

In [ ]:
def coord_caixa(caixa):
  (te, td, bd, be) = caixa
  te = (int(te[0]), int(te[1]))
  td = (int(td[0]), int(td[1]))
  bd = (int(bd[0]), int(bd[1]))
  be = (int(be[0]), int(be[1]))
  return te, td, bd, be

def desenha_caixa(img, te, bd, cor_caixa=(200, 255, 0), espessura=2):
  cv2.rectangle(img, te, bd, cor_caixa, espessura)
  return img

In [ ]:
nome_arquivo = 'resultado_easy.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [ ]:
amostras_exibir = 20
amostra_atual = 0

In [ ]:
cor_fonte = (0,0,0)
cor_fundo = (200,255,0)
cor_caixa = (200,255,0)
tam_fonte = 20

In [ ]:
while (cv2.waitKey(1) < 0):
    conectado, frame = cap.read()

    if not conectado:
        break

    frame = cv2.resize(frame, (video_largura, video_altura))

    imagem_cp = frame.copy()

    reader = Reader(idiomas, gpu=gpu)
    resultados = reader.readtext(frame)

    for (caixa, texto, prob) in resultados:
      te, td, bd, be = coord_caixa(caixa)

      frame = desenha_caixa(frame, te, bd)
      frame = fundo_texto(texto, te[0], te[1], frame, fonte, tam_fonte, cor_fundo)
      frame = escreve_texto(texto, te[0], te[1], frame, fonte, cor_fonte, tam_fonte)

    if amostra_atual <= amostras_exibir:
      cv2_imshow(frame)
      amostra_atual = amostra_atual + 1

    saida_video.write(frame)

print("Terminou")
saida_video.release()
cv2.destroyAllWindows()